In [228]:
G = 4.3009E-3 *1u"pc *(km/s)^2 * Msun"
function format_parameters(galaxy_args)
    #I didn't have this at first, but this makes all galaxies have an uniform format. Not too crazy
    return Dict(
        "mass" => galaxy_args[1]*1u"Msun",
        "radius"     => galaxy_args[2]*1u"pc",
        "center_pos" => galaxy_args[3].*1u"pc",
        "center_vel" => galaxy_args[4].*1u"km/s",
        "normal"     => galaxy_args[5],
        "Nᵣ"    => galaxy_args[6],
        "Nₛ"    => galaxy_args[7],
        "softening"  => galaxy_args[8]
    )
end
function init_disk!(galaxy,dt=1E-4u"yr")
    #=
    This function takes a 'galaxy' as an argument, which is an array of 8 arguments=>
        [mass,radius,center_pos,center_vel,normal,N_rings,N_stars,softening]
    And outputs the star positions, velocities and the velocity scale.
    =#
    dr = (1-galaxy["softening"]) * galaxy["radius"]/galaxy["Nᵣ"] # Disregarding the softening, this is just the width of the rings(raidus/number of slices)
    StarsPerRing = trunc(Int64,galaxy["Nₛ"]/galaxy["Nᵣ"])
#-------------------------------------------------------------Rotations--------------------------------------------------------------
    if norm(galaxy["normal"]) == 0 
        Rotation = I
    else 
        cosθ = normalize(galaxy["normal"])[3]
        sinθ = √(1-cosθ^2)
        u = [0,0,1] × normalize(galaxy["normal"])
        u = normalize(u)

        Rotation = [
        u[1]*u[1]*(1-cosθ)+cosθ u[1]*u[2]*(1-cosθ)-u[2]*sinθ u[1]*u[3]*(1-cosθ)+u[1]*sinθ;

        u[2]*u[1]*(1-cosθ)+u[3]*sinθ u[2]*u[2]*(1-cosθ)+cosθ u[2]*u[3]*(1-cosθ)-u[1]*sinθ;

        u[3]*u[1]*(1-cosθ)+u[2]*sinθ u[3]*u[1]*(1-cosθ)+u[1]*sinθ u[3]*u[3]*(1-cosθ)+cosθ
        ]
    end
#-----------------------------------------------------------------------------------------------------------------------------------
    galaxy["stars_pos"] = []
    galaxy["stars_vel"] = []

    R = galaxy["softening"] * galaxy["radius"]
    for i ∈ 1:galaxy["Nᵣ"] #iterate over the rings
        #Randomly distribute the initial radii and angles 
        rₛ =(R * ones(StarsPerRing) + dr *rand(StarsPerRing))./1u"pc" #creates an n-dimensional vector of radii from R to R+dr 
        ϕₛ = 2π *rand(StarsPerRing)

        # Positions 
        vecᵣ = (Rotation * [rₛ.*cos.(ϕₛ),rₛ.*sin.(ϕₛ),zeros(StarsPerRing)]/.1u"pc").*1u"pc"
        x = galaxy["center_pos"][1].+vecᵣ[1]
        y = galaxy["center_pos"][2].+vecᵣ[2]
        z = galaxy["center_pos"][3].+vecᵣ[3]

        # Velocities
        Tₛ = 2π *√(((rₛ*1u"pc")^3)/(G *galaxy["mass"]))
        Δϕ = 2π *dt/Tₛ

        vecᵥ = uconvert.(u"km/s",Rotation* [rₛ.*(cos.(ϕₛ)-cos.(ϕₛ-Δϕ)),rₛ.*(sin.(ϕₛ)-sin.(ϕₛ-Δϕ)),zeros(StarsPerRing)])
        v₁ = galaxy["center_vel"][1].+vecᵥ[1]
        v₂ = galaxy["center_vel"][2].+vecᵥ[2]
        v₃ = galaxy["center_vel"][3].+vecᵥ[3]
        push!(galaxy["stars_pos"],[x,y,z])
        push!(galaxy["stars_vel"],[v₁,v₂,v₃])
        R += dr
    end 
    galaxy["vel_scale"]=uconvert(u"km/s",√(G*galaxy["mass"]/(0.5*R)))
end

init_disk! (generic function with 2 methods)

In [229]:
Introoder = format_parameters([5e10, 10, [-5,5,1], [15,-15,0], [1,-1,2^0.5], 10, 4000, 0.025])
init_disk!(Introoder)

Unitful.DimensionError: DimensionError: -5.0 pc and -0.7584521895523385 are not dimensionally compatible.

In [230]:
cosθ = normalize(Introoder["normal"])
Rotation = rand(3,3)
R = Introoder["softening"] * Introoder["radius"]
dr = (1-Introoder["softening"]) * Introoder["radius"]/Introoder["Nᵣ"] # Disregarding the softening, this is just the width of the rings(raidus/number of slices)
StarsPerRing = trunc(Int64,Introoder["Nₛ"]/Introoder["Nᵣ"])
rₛ = (R * ones(StarsPerRing) + dr *rand(StarsPerRing))./1u"pc" #creates an n-dimensional vector of radii from R to R+dr 
ϕₛ = 2π *rand(StarsPerRing)
cosθ = normalize(Introoder["normal"])[3]
        sinθ = √(1-cosθ^2)
        u = [0,0,1] × normalize(Introoder["normal"])
        u = normalize(u)

        Rotation = [
        u[1]*u[1]*(1-cosθ)+cosθ u[1]*u[2]*(1-cosθ)-u[2]*sinθ u[1]*u[3]*(1-cosθ)+u[1]*sinθ;

        u[2]*u[1]*(1-cosθ)+u[3]*sinθ u[2]*u[2]*(1-cosθ)+cosθ u[2]*u[3]*(1-cosθ)-u[1]*sinθ;

        u[3]*u[1]*(1-cosθ)+u[2]*sinθ u[3]*u[1]*(1-cosθ)+u[1]*sinθ u[3]*u[3]*(1-cosθ)+cosθ
        ]
        vecᵣ =Rotation.*[rₛ.*cos.(ϕₛ),rₛ.*sin.(ϕₛ),zeros(StarsPerRing)].*1u"pc"
        Introoder["center_pos"][1] .+ vecᵣ[1]

400-element Vector{Quantity{Float64, 𝐋, Unitful.FreeUnits{(pc,), 𝐋, nothing}}}:
  -5.094684499654795 pc
  -5.484212915555774 pc
   -4.51076546668597 pc
  -5.471847912064625 pc
  -4.575683851938968 pc
  -5.150629462097826 pc
 -5.0587989464723835 pc
  -4.655217153946315 pc
  -5.172851477527959 pc
  -4.678461059757543 pc
                      ⋮
  -5.286653402633439 pc
 -5.3649220121213474 pc
  -5.463794937462642 pc
   -5.13902548581043 pc
  -5.022471192137517 pc
  -5.920634150498412 pc
  -4.542048292229277 pc
  -4.663571701080697 pc
 -4.4255128212141015 pc

In [58]:
a=[0,0,4]
b=[1,2,3]
a×b

MethodError: MethodError: no method matching identity(::Int64, ::Int64)
Closest candidates are:
  identity(::Any) at operators.jl:526

UniformScaling{Int64}
3*I